In [2]:
import torch
from torchvision import transforms
import matplotlib.pyplot as plt
# from torch.utils.tensorboard import SummaryWriter
import warnings

warnings.simplefilter("ignore")
test_transform = transforms.Compose([
        transforms.Resize((224,224)),
#         transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
model = torch.load('iterdrop48.pth', map_location=torch.device('cpu'))
from PIL import Image
classes = ['aadhar', 'college_id', 'pancard', 'personal_photo']
def predict(model, test_image_name):

    transform = test_transform

    test_image = Image.open(test_image_name).convert('RGB')
    plt.imshow(test_image)

    test_image_tensor = transform(test_image)
#     print(test_image_tensor.size())

    if torch.cuda.is_available():
        test_image_tensor = test_image_tensor.view(1, 3, 224, 224).cuda()
    else:
        test_image_tensor = test_image_tensor.view(1, 3, 224, 224)

    with torch.no_grad():
        model.eval()
        # Model outputs log probabilities
        out = model(test_image_tensor)
        _ , b = (torch.max(out,1))
#         print(out)
#         print(train_data_loader.dataset.classes[b.item()])
        ps = torch.exp(out)
        topk, topclass = ps.topk(1, dim=1)
#     print(ps)
#     print(topk,topclass)
    return classes[b.item()], topk.cpu().numpy()[0][0]

In [3]:
import os
import shutil
curr_path="/home/sidharth/Documents/datasets/students_id"
copy_coll_path="/home/sidharth/Documents/verify_id/app/result/college_id"
copy_pan_path="/home/sidharth/Documents/verify_id/app/result/pan"
copy_aadhar_path="/home/sidharth/Documents/verify_id/app/result/aadhar"
copy_personal_path="/home/sidharth/Documents/verify_id/app/result/personal"
count=0
per=0
for file in os.listdir(curr_path):
    count+=1
    
    if count<10213:
        continue
    print(file)
    result=predict(model,os.path.join(curr_path, file))
    
    
    """if result[0]=='college_id'and result[1]>=0.99:
        shutil.move(os.path.join(curr_path, file), os.path.join(copy_coll_path, file))
    elif result[0]=='aadhar' and result[1]>=0.90:
        shutil.move(os.path.join(curr_path, file),os.path.join(copy_aadhar_path, file))
    elif result[0]=='pancard'and result[1]>=0.90:
        shutil.move(os.path.join(curr_path, file),os.path.join(copy_pan_path, file))"""
    if result[0]=='personal_photo' and result[1]>=0.90:
        per+=1
        shutil.copy(os.path.join(curr_path, file),os.path.join(copy_personal_path, file))     
    print(count, per)    

        

4rSAp3wr5H.jpg
10213 0
YjsMZAwz4X.jpg
10214 1
L8RoIhQOyU.jpg
10215 1
IOYGuNKb5C.jpg
10216 2
p7psSgCvfk.jpg
10217 3
1zKXuOQHhm.png
10218 3
cfqpgsxwkC.jpg
10219 4
UYeN6Wo6FU.jpg
10220 4
quSHoW0TmK.jpg
10221 4
KlSbmzbWe7.jpeg
10222 4
YjhyZotSLh.jpg
10223 4
ivagejRkom.png
10224 4
flB6s2vG5t.png
10225 4
1q4kmtHtpx.jpeg
10226 4
0qsvV4tBQf.png
10227 4
zRvVObs9U2.jpg
10228 4
OUzz2beyRK.jpg
10229 4
Kk1jy85n1s.jpeg
10230 4
5TSTXs8hxO.jpg
10231 4
nPK68zLmZ5.jpg
10232 4
8n5TUWEoWG.jpg
10233 4
T1vupadFtQ.jpg
10234 4
nwfydyekFb.jpg
10235 4
Ty0svkD8WF.jpeg
10236 4
oHw5HDtsc9.jpg
10237 4
G17g12gGr2.jpeg
10238 4
5tHRR62GHI.jpg
10239 4
N9EOrwLiag.jpg
10240 5
eFbiLW0gjv.jpg
10241 5
VDrHe0c8ex.jpg
10242 5
jw5vKTWuyp.jpg
10243 5
x8sS8WbdW0.jpg
10244 5
9ifnWo1knZ.jpeg
10245 5
jzZ2GYKZIk.jpg
10246 5
EERmFTNz2o.jpg
10247 5
xIh8eqAKAR.png
10248 5
MUQ7SlDPHS.jpg
10249 5
70FRFbswh6.jpg
10250 5
GYPsadyiKZ.png
10251 5
e5zQZVq3n0.jpg
10252 5
aZNGOR0WJ0.jpg
10253 6
j4q7iTl1jB.jpg
10254 7
aIxAxWLfUm.jpeg
10255 7
K5oC

KeyboardInterrupt: 